## Webscrapping

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:

# URL of the flipcrt search results page
url = 'https://www.flipkart.com/home-furnishing/bed-linen-blankets/bedsheets/pr?sid=jra%2Cknw%2Cqcw&marketplace=FLIPKART&otracker=nmenu_sub_Home+%26+Furniture_0_Bedsheets&p%5B%5D=facets.rating%255B%255D%3D1%25E2%2598%2585%2B%2526%2Babove'

In [3]:
def switch_tab(source,tab_index):
    link = source.get('href') #getting the URL to open in a new tab.,with html attribut href which specifies destination of link included in <a> tag
    driver.execute_script(f'window.open("{link}", "_blank");') #opening new tab with URL from previous step.

    # Wait for the new window to open and then switch to it
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > tab_index)
    driver.switch_to.window(driver.window_handles[tab_index])

    # Wait for the new tab to load its content,until HTML element with the class name 'fonts-loaded' is present on the page.
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'fonts-loaded'))
    )

    # Getting the HTML content of the new tab(current page)
    new_tab_content = driver.page_source
    soup = BeautifulSoup(new_tab_content, 'html.parser')#creating a BeautifulSoup object which can be used to parse the HTML and extract data from it

    return soup

In [4]:
# Set up the Selenium WebDriver
driver = webdriver.Chrome()
driver.maximize_window()

# Load the page
driver.get(url)

# Wait for the page to fully load
time.sleep(5)

# Get the HTML content of the page
html_content = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find elements containing specific div information
products = soup.find_all('div', class_='slAVV4', limit=20)

data = []


for product in products:
    product_details = {} #creating an empty dictionary to store the product details for each product.
    name = product.find('a', class_='wjcEIp')
    name = name.text if name else "No name available"

    # Extract the price of the product
    price = product.find('div', class_='Nx9bqj')
    price = price.text if price else "No price available"

    # Extract the rating of the product
    rating = product.find('div', class_='XQDdHH')
    rating = rating.text if rating else "No rating available"


# #Extract number of ratings and reviews
#     num_ratings_reviews_container = product.find('span', class_='Wphh3N')
#     if num_ratings_reviews_container:
#         ratings_reviews_text = num_ratings_reviews_container.get_text().strip()
        
        
#         # Split the text to isolate ratings and reviews
#         ratings_text, reviews_text = ratings_reviews_text.split('&')
        
#         # Extract just the numbers
#         num_ratings = ''.join(filter(str.isdigit, ratings_text))
#         num_reviews = ''.join(filter(str.isdigit, reviews_text))
#     else:
#         num_ratings = "No ratings"
#         num_reviews = "No reviews"


# Extract number of ratings and reviews
    num_ratings_reviews_container = product.find('span', class_='Wphh3N')
    if num_ratings_reviews_container:
        # Find all span elements within the container
        span_elements = num_ratings_reviews_container.find_all('span')

        # Check if there are at least 3 span elements (for ratings, '&', and reviews)
        if len(span_elements) >= 3:
            # Extract the text from the first and third span elements
            ratings_text = span_elements[0].get_text().strip()
            reviews_text = span_elements[2].get_text().strip()

            # Extract just the numbers
            num_ratings = ''.join(filter(str.isdigit, ratings_text))
            num_reviews = ''.join(filter(str.isdigit, reviews_text))
        else:
            num_ratings = "No ratings"
            num_reviews = "No reviews"
    else:
        num_ratings = "No ratings"
        num_reviews = "No reviews"

#--------------------------6 goshti print karacha code kadhun ahe ithun
    
    product_details['Product Name'] = name  #storing the extracted details in the product_details dictionary
    product_details['Price'] = price
    product_details['Rating'] = rating
    product_details['Number of Ratings'] = num_ratings
    product_details['Number of Reviews'] = num_reviews
    
    if num_reviews != "No reviews" and num_reviews != "0":
        soup = switch_tab(product,1)     #If the product has reviews, it switches to the reviews tab

        # Find the element with the reviews or link to reviews
        div_element = soup.find('div', class_='col JOpGWq')

        if div_element:
            # Find all <a> tags within the div
            all_a_tags = div_element.find_all('a') #-------------------------------------
            # Check if there are any <a> tags
            if len(all_a_tags) > 10:
                # Get the last <a> tag
                last_a_tag = all_a_tags[-1]

                soup = switch_tab(last_a_tag,2) # finding the container with the reviews, extracts the summary and details of each review, and stores them in the product_details dictionary.
                #If there are more than 10 reviews, it opens another tab to the page with the remaining reviews and extracts those as well.

                reviews_container = soup.find_all('div',class_='col _2wzgFH K0kLPL')
                review_summary_list = []
                review_details_list = []
                for review in reviews_container:
                    review_summary = review.find('p',class_='_2-N8zT').text if review.find('p',class_='_2-N8zT') else "No summary"
                    review_details = review.find('div',class_='t-ZTKy').text if review.find('div',class_='t-ZTKy') else "No details"
                    review_summary_list.append(review_summary)
                    review_details_list.append(review_details)
                    # print(f"{reviews_container.index(review)}. review_summary: {review_summary}")
                    # print(f"{reviews_container.index(review)}. review_details: {review_details}")
                product_details['Review Summary'] = review_summary_list
                product_details['Review Details'] = review_details_list
                driver.close()
            else:
                review_summary_list = []
                review_details_list = []
                reviews_summary_container = soup.find_all('p',class_='_2-N8zT')
                for reviews_summary in reviews_summary_container:
                    reviews_summary = reviews_summary.text
                    review_summary_list.append(reviews_summary)
                #     # print(f"{reviews_summary_container.index(reviews_summary)}. review_summary: {review_summary}")
                reviews_details_container = soup.find_all('div',class_='t-ZTKy')
                for review_details in reviews_details_container:
                    # print(reviews_details_container.index(review_details))
                    review_details = review_details.text
                    review_details_list.append(review_details)
                    # print(f". review_details: {review_details}")
                product_details['Review Summary'] = review_summary_list
                product_details['Review Details'] = review_details_list

            # print('-' *50)
            
            driver.switch_to.window(driver.window_handles[1])
            driver.close()    #Close the reviews tab
        driver.switch_to.window(driver.window_handles[0])
    data.append(product_details) #It adds the product_details dictionary to the data list.

# Close the browser
driver.quit()

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=124.0.6367.207)
Stacktrace:
	GetHandleVerifier [0x00007FF632411522+60802]
	(No symbol) [0x00007FF63238AC22]
	(No symbol) [0x00007FF632247CE4]
	(No symbol) [0x00007FF6322405B3]
	(No symbol) [0x00007FF6322313E4]
	(No symbol) [0x00007FF632232C14]
	(No symbol) [0x00007FF632231711]
	(No symbol) [0x00007FF632231049]
	(No symbol) [0x00007FF632230D42]
	(No symbol) [0x00007FF63222EBE4]
	(No symbol) [0x00007FF63222F22C]
	(No symbol) [0x00007FF63224A9F9]
	(No symbol) [0x00007FF6322DAB7E]
	(No symbol) [0x00007FF6322BAB7A]
	(No symbol) [0x00007FF6322DA224]
	(No symbol) [0x00007FF6322BA923]
	(No symbol) [0x00007FF632288FEC]
	(No symbol) [0x00007FF632289C21]
	GetHandleVerifier [0x00007FF6327141BD+3217949]
	GetHandleVerifier [0x00007FF632756157+3488183]
	GetHandleVerifier [0x00007FF63274F0DF+3459391]
	GetHandleVerifier [0x00007FF6324CB8E6+823622]
	(No symbol) [0x00007FF632395FBF]
	(No symbol) [0x00007FF63239

In [ ]:
data

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
# df.to_csv(r'C:\Users\acer\Desktop\ds class\nlp\sentiment analysis project amazon\scraped_data1.csv', index=False, encoding='utf-8-sig')